In [1]:
import pandas as pd
import numpy as np
import datetime as dt

### Identify the new and returning customers

In [10]:
# obtaining the data

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
orders = pd.read_csv('./Data/KOI_export_082021.csv')

C:\Users\Angela\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (28,38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
orders['Financial Status'].unique()

array(['paid', nan, 'partially_refunded', 'refunded', 'pending'],
      dtype=object)

In [6]:
orders_1 = pd.read_csv('./Data/orders_export_1.csv')
orders_2 = pd.read_csv('./Data/orders_export_2.csv')

In [7]:
orders_com = orders_1.append(orders_2, ignore_index=True)

In [8]:
orders_com['Financial Status'].unique()

array(['paid', nan, 'refunded', 'partially_refunded', 'pending', 'voided'],
      dtype=object)

In [5]:
orders['Order_Date'] = pd.to_datetime(orders['Created at'], format="%Y-%m-%d %H:%M:%S %z", utc=True)
orders['Date'] = pd.to_datetime(orders['Order_Date'].dt.date)
orders['Date_YM'] = orders['Date'].apply(lambda x: x.strftime('%Y-%m'))

In [19]:
orders_com['Order_Date'] = pd.to_datetime(orders_com['Created at'], format="%Y-%m-%d %H:%M:%S %z", utc=True)
orders_com['Date'] = pd.to_datetime(orders_com['Order_Date'].dt.date)
orders_com['Date_YM'] = orders_com['Date'].apply(lambda x: x.strftime('%Y-%m'))

In [7]:
df = orders[['Date','Name','Total', 'Financial Status','Email', 'Subtotal', 'Shipping',
       'Taxes', 'Discount Amount', 'Refunded Amount', 'Billing Country']]
#df_com = orders_com[['Date','Name','Total', 'Financial Status','Email', 'Subtotal', 'Shipping',
       #'Taxes', 'Discount Amount', 'Refunded Amount', 'Billing Country']]

In [ ]:
df_com.rename(columns={'Date': 'date','Name':'order_id','Total':'total', 'Financial Status':'status','Email':'email', 'Subtotal':'subtotal', 'Shipping':'shipping',
       'Taxes':'taxes', 'Discount Amount':'discount_amount', 'Refunded Amount':'refunded_amount', 'Billing Country': 'billing_country'}, inplace=True)

In [8]:
df.rename(columns={'Date': 'date','Name':'order_id','Total':'total', 'Financial Status':'status','Email':'email', 'Subtotal':'subtotal', 'Shipping':'shipping',
       'Taxes':'taxes', 'Discount Amount':'discount_amount', 'Refunded Amount':'refunded_amount', 'Billing Country': 'billing_country'}, inplace=True)

C:\Users\Angela\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [9]:
#df_com.shape, 
df.shape

(13644, 11)

In [2]:
# Customers

In [10]:
%run ./cohorts_pipeline_koi.py

In [11]:
#df_com = first_order(df_com)
df = first_order(df)

In [12]:
#df_com.shape, 
df.shape

(6404, 17)

In [86]:
df_com.head(2)

,email,date,order_id,total,status,subtotal,shipping,taxes,discount_amount,refunded_amount,billing_country,order_date,first_order,first_order_ym,date_ym,customer_type,days_between
0,ernst.hiemstra@gmail.com,2018-08-09,KOI1002,84.97,paid,84.97,0.0,14.75,0.0,0.0,NL,2018-08-09,2018-08-09,2018-08,2018-08,New,0.0
1,andre@kingsofindigo.com,2018-08-14,KOI1003,184.92,paid,184.92,0.0,32.09,0.0,0.0,NL,2018-08-14,2018-08-14,2018-08,2018-08,New,0.0


In [87]:
df_com.isnull().sum()

email              12
date                0
order_id            0
total               0
status              0
subtotal            0
shipping            0
taxes               0
discount_amount     0
refunded_amount     0
billing_country    86
order_date          0
first_order        12
first_order_ym     12
date_ym             0
customer_type       0
days_between        0
dtype: int64

In [13]:
df.head(2)

,email,date,order_id,total,status,subtotal,shipping,taxes,discount_amount,refunded_amount,billing_country,order_date,first_order,first_order_ym,date_ym,customer_type,days_between
0,andrekortekaas@hotmail.com,2019-12-04,20001001,8.20,paid,3.25,4.95,0.56,29.24,0.0,NL,2019-12-04,2019-12-04,2019-12,2019-12,New,0.0
1,andrekortekaas@hotmail.com,2019-12-10,20001002,24.95,paid,20.00,4.95,3.47,179.95,0.0,NL,2019-12-10,2019-12-04,2019-12,2019-12,Returning,6.0


In [14]:
df.isnull().sum()

email              0
date               0
order_id           0
total              0
status             0
subtotal           0
shipping           0
taxes              0
discount_amount    0
refunded_amount    0
billing_country    0
order_date         0
first_order        0
first_order_ym     0
date_ym            0
customer_type      0
days_between       0
dtype: int64

In [91]:
df_com[df_com['date']> '2020-01-01']['customer_type'].value_counts()

New          7260
Returning    4098
Name: customer_type, dtype: int64

In [92]:
df[df['date']> '2020-01-01']['customer_type'].value_counts()

New          3543
Returning     628
Name: customer_type, dtype: int64

In [95]:
df_com[df_com['date']> '2021-02-01']['subtotal'].sum() + df[df['date']> '2021-02-01']['subtotal'].sum()

190115.4

In [101]:
summary_de = df.groupby(['date_ym', 'billing_country', 'customer_type']).agg({'subtotal':pd.Series.sum, 'order_id':pd.Series.count, 'shipping':pd.Series.sum, 'taxes':pd.Series.sum, 'total':pd.Series.sum, 'discount_amount':pd.Series.sum, 'refunded_amount':pd.Series.sum}).reset_index()
summary_com = df_com.groupby(['date_ym', 'billing_country', 'customer_type']).agg({'subtotal':pd.Series.sum, 'order_id':pd.Series.count, 'shipping':pd.Series.sum, 'taxes':pd.Series.sum, 'total':pd.Series.sum, 'discount_amount':pd.Series.sum, 'refunded_amount':pd.Series.sum}).reset_index()

In [114]:
summary_com[summary_com['date_ym']== '2021-02'][summary_com['billing_country'].isin(['DE', 'AT', 'CH'])]['subtotal'].sum() + summary_de[summary_de['date_ym']== '2021-02'][summary_de['billing_country'].isin(['DE', 'AT', 'CH'])]['subtotal'].sum()

<ipython-input-114-f72d2f490e54>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  summary_com[summary_com['date_ym']== '2021-02'][summary_com['billing_country'].isin(['DE', 'AT', 'CH'])]['subtotal'].sum() + summary_de[summary_de['date_ym']== '2021-02'][summary_de['billing_country'].isin(['DE', 'AT', 'CH'])]['subtotal'].sum()


77939.53

In [115]:
78159.51 - 77939.53

219.97999999999593

In [118]:
summary_de['TO_EXSHIP_EXVAT'] = summary_de['subtotal'] - summary_de['taxes']
summary_com['TO_EXSHIP_EXVAT'] = summary_com['subtotal'] - summary_com['taxes']

In [119]:
summary_de['TO_EXSHIP_EXVAT_EXRET'] = summary_de['TO_EXSHIP_EXVAT'] - summary_de['refunded_amount']
summary_com['TO_EXSHIP_EXVAT_EXRET'] = summary_com['TO_EXSHIP_EXVAT'] - summary_com['refunded_amount']

In [120]:
summary_de.to_clipboard(decimal=',', index=False)

In [121]:
summary_com.to_clipboard(decimal=',', index=False)

In [37]:
# Cohorts

In [66]:
transactions, cohorts = cohorts_pipeline(df)

In [68]:
try:
    transactions =  transactions.astype(int) 
except ValueError:
    pass

In [69]:
transactions.isna().sum()

New_Trans    0
Ret_Trans    1
New_Rev      0
Ret_Rev      0
AOV_New      0
AOV_Ret      1
dtype: int64

In [70]:
transactions.fillna(0)

,New_Trans,Ret_Trans,New_Rev,Ret_Rev,AOV_New,AOV_Ret
date_ym,,,,,,
2018-11,42.0,0.0,10149.19,0.00,241.647381,0.000000
2018-12,172.0,8.0,32490.85,1250.55,188.900291,156.318750
2019-01,416.0,47.0,72944.71,6028.86,175.347861,128.273617
2019-02,336.0,47.0,59669.68,7711.35,177.588333,164.071277
2019-03,253.0,84.0,43641.70,11784.78,172.496838,140.295000
2019-04,148.0,41.0,27784.62,6902.64,187.733919,168.357073
2019-05,151.0,45.0,31647.92,8227.36,209.588874,182.830222
2019-06,136.0,44.0,24815.29,6663.05,182.465368,151.432955
2019-07,249.0,90.0,36061.30,14093.44,144.824498,156.593778


In [40]:
transactions.to_clipboard()

In [71]:
cohorts = cohorts.astype(str)

In [72]:
cohorts = cohorts.apply(lambda x: x.str.replace('.',','))
cohorts = cohorts.apply(lambda x: x.str.replace('nan',''))

In [ ]:
cohorts.to_clipboard()